<a href="https://colab.research.google.com/github/Venukumar975/YOLOv8n-Android-Detection/blob/main/YOLO-Training-On-Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00


In [ ]:
yaml_content = """
# Ultralytics YOLO 🚀, AGPL-3.0 license
# Custom Brain Architecture for PASCAL VOC

nc: 20  # number of classes
scales: # model compound scaling constants, i.e. 'n', 's', 'm', 'l', 'x'
  n: [0.33, 0.25, 1024]

backbone:
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, true]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, C2f, [256, true]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 6, C2f, [512, true]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, true]]
  - [-1, 1, SPPF, [1024, 5]]  # 9

head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [512]]  # 12

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [256]]  # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]]  # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]  # cat head P5
  - [-1, 3, C2f, [1024]]  # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]]  # Detect(P3, P4, P5)
"""

with open("my_custom_brain.yaml", "w") as f:
    f.write(yaml_content)
print("✅ 'my_custom_brain.yaml' created successfully!")

✅ 'my_custom_brain.yaml' created successfully!


In [ ]:
from ultralytics import YOLO
from google.colab import drive
import os

drive.mount('/content/drive')

drive_folder = "/content/drive/MyDrive/YOLO_Training_Runs"
os.makedirs(drive_folder, exist_ok=True)

print(f"✅ Drive Mounted! Checkpoints will be saved to: {drive_folder}")

model = YOLO('my_custom_brain.yaml')

results = model.train(
    data='VOC.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    name='voc_scratch_safe',
    project=drive_folder,
    save_period=1
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive
✅ Drive Mounted! Checkpoints will be saved to: /content/drive/MyDrive/YOLO_Training_Runs
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=VOC.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=

KeyboardInterrupt: 

In [ ]:

nc: 20

"""
names:
  0: aeroplane
  1: bicycle
  2: bird
  3: boat
  4: bottle
  5: bus
  6: car
  7: cat
  8: chair
  9: cow
  10: diningtable
  11: dog
  12: horse
  13: motorbike
  14: person
  15: pottedplant
  16: sheep
  17: sofa
  18: train
  19: tvmonitor
"""


In [ ]:
import os
from ultralytics import YOLO
from google.colab import drive


drive.mount('/content/drive')

drive_folder = "/content/drive/MyDrive/YOLO_Training_Runs"
project_name = "voc_scratch_safe"

last_weight_path = f"{drive_folder}/{project_name}/weights/last.pt"


if os.path.exists(last_weight_path):
    print(f"✅ Found interrupted training at: {last_weight_path}")
    print("🔄 Resuming training from the last saved epoch...")

    model = YOLO(last_weight_path)


    model.train(resume=True)

else:
    print("🚀 No previous training found. Starting FRESH training...")

    model = YOLO('my_custom_brain.yaml')

    model.train(
        data='VOC.yaml',
        epochs=63,
        imgsz=640,
        batch=16,
        project=drive_folder,
        name=project_name,
        save_period=1,
        exist_ok=True
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Found interrupted training at: /content/drive/MyDrive/YOLO_Training_Runs/voc_scratch_safe/weights/last.pt
🔄 Resuming training from the last saved epoch...
Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/usr/local/lib/python3.12/dist-packages/ultralytics/cfg/datasets/VOC.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=Fa

KeyboardInterrupt: 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1. Compressing '/content/datasets/VOC'... (This may take a minute)
2. Moving compressed file to Google Drive...
✅ Success! Your dataset is saved at: /content/drive/MyDrive/voc_backup.tar.gz


In [ ]:
!ls -R /content/VOC_extracted/

Streaming output truncated to the last 5000 lines.
2007_009832.jpg  2008_008624.jpg  2010_002990.jpg  2011_004485.jpg
2007_009841.jpg  2008_008627.jpg  2010_002991.jpg  2011_004486.jpg
2007_009889.jpg  2008_008628.jpg  2010_002993.jpg  2011_004487.jpg
2007_009897.jpg  2008_008629.jpg  2010_002995.jpg  2011_004488.jpg
2007_009899.jpg  2008_008632.jpg  2010_003002.jpg  2011_004489.jpg
2007_009901.jpg  2008_008635.jpg  2010_003003.jpg  2011_004490.jpg
2007_009910.jpg  2008_008636.jpg  2010_003007.jpg  2011_004491.jpg
2007_009911.jpg  2008_008637.jpg  2010_003010.jpg  2011_004494.jpg
2007_009923.jpg  2008_008641.jpg  2010_003011.jpg  2011_004495.jpg
2007_009938.jpg  2008_008642.jpg  2010_003013.jpg  2011_004500.jpg
2007_009947.jpg  2008_008649.jpg  2010_003014.jpg  2011_004503.jpg
2007_009950.jpg  2008_008652.jpg  2010_003015.jpg  2011_004505.jpg
2008_000002.jpg  2008_008654.jpg  2010_003016.jpg  2011_004506.jpg
2008_000003.jpg  2008_008658.jpg  2010_003017.jpg  2011_004509.jpg
2008_000007

In [ ]:
import os
from ultralytics import YOLO

EXTRACT_DIR = '/content/VOC_extracted'
BASE_RUNS_DIR = '/content/drive/MyDrive/YOLO_Training_Runs/voc_scratch_safe/weights'
WEIGHTS_FILE = 'best.pt'
MODEL_PATH = os.path.join(BASE_RUNS_DIR, WEIGHTS_FILE)

DATA_YAML_PATH = os.path.join(EXTRACT_DIR, 'data.yaml')

print(f"Loading model from: {MODEL_PATH}")
model = YOLO(MODEL_PATH)

print(f"\n--- Running TEST Set Validation with data path: {DATA_YAML_PATH} ---")

if not os.path.exists(DATA_YAML_PATH):
    print("FATAL ERROR: The data.yaml file was not found at the assumed path. Please manually check /content/VOC_extracted/.")
    exit()

metrics = model.val(
    data=DATA_YAML_PATH,
    split='test',
    save_json=True,
    plots=True
)

print("\n--- Summary Metrics on TEST Set ---")
print(f"mAP50-95 (Overall): {metrics.box.map}")
print(f"mAP50 (Overall): {metrics.box.map50}")
print(f"mAP75 (Overall): {metrics.box.map75}")

print("\n--- Next Step ---")
print("Check the 'runs/detect/' folder for the 'results.json' file to get the Precision, Recall, and mAP for all 20 VOC classes.")